In [1]:
import environment.job_search_environment as job_search_env
import argparse
import os
import random
from supersuit import pad_observations_v0, pad_action_space_v0

import ray
from ray import tune
from ray.tune.registry import register_env

In [2]:
# NOTE: This is working on the Zoo, but not on my local machine (M1 compatibility issues)
from ray.rllib.algorithms.ppo import PPOConfig

In [4]:
import gym

In [5]:
from ray import tune
from ray.rllib.models.modelv2 import restore_original_dimensions
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.tune.registry import register_env
import torch
from torch import nn
from ray.rllib.utils.framework import try_import_tf, try_import_torch

In [6]:
tf1, tf, tfv = try_import_tf()
torch, _ = try_import_torch()

In [7]:
def env_creator(args):
    env = job_search_env.env()
    return env

In [8]:
env_name = "job_search"
register_env(env_name, lambda config: ParallelPettingZooEnv(env_creator(config)))

In [9]:
ray.shutdown()

In [10]:
ray.init()

E1212 21:59:20.825869883 1670454 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E1212 21:59:20.848930613 1670454 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E1212 21:59:22.280175257 1670454 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E1212 21:59:22.299151579 1670454 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
2022-12-12 21:59:22,421	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.10.8
Ray version:,2.1.0


In [11]:
"""
In order to deal with the Dictionary space, need to pass a custom model to RLlib.
See: https://medium.com/@nima.siboni/rllib-with-dictionary-state-baa06b64470f
"""
class CandidateModelV0(TorchModelV2, nn.Module):
    def __init__(self, obs_space, act_space, num_outputs, *args, **kwargs):
        TorchModelV2.__init__(self, obs_space, act_space, num_outputs, *args, **kwargs)
        nn.Module.__init__(self)
#         self.layer1 = torch.cat()
        self.next_layers = nn.Sequential(
            nn.Linear(3136, 512),
            nn.ReLU(),
        )
        
        self.policy_fn = nn.Linear(512, num_outputs)
        self.value_fn = nn.Linear(512, 1)
        
    def forward(self, input_dict, state, seq_lens):
        model_out = self.model(input_dict["obs_flat"])

In [12]:
ModelCatalog.register_custom_model("CandidateModelV0", CandidateModelV0)

In [13]:
# TODO: use policy_map to map different policies to candidate and employer agents

In [14]:
config = (
    PPOConfig()
    .environment(env=env_name, clip_actions=True)
    .debugging(log_level="DEBUG")
    .framework(framework="torch")
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
).to_dict()

In [15]:
isinstance(job_search_env.env().observation_space("candidate_0"), gym.spaces.Dict)

True

In [16]:
config["observation_space"] = job_search_env.env().observation_space("candidate_0")
config["action_space"] = job_search_env.env().action_space("candidate_0")
# There's something wrong with my observation space, not able to be flattened by the Dict FlatteningPreprocessor
# config["observation_space"] = Dict({"test1": Discrete(10)})
# config["action_space"] = Tuple((Discrete(3), Discrete(10)))
print (config["observation_space"])


Dict(candidate_obs:Dict(accepted_offer:Dict(employer_0:Discrete(101)), counter_offers:Dict(employer_0:Tuple(Discrete(101), Discrete(11))), current_offers:Dict(employer_0:Tuple(Discrete(101), Discrete(11))), job_openings:Dict(employer_0:Discrete(2)), rejected_offers:Dict(employer_0:Tuple(Discrete(2), Discrete(101)))), employer_obs:Dict(accepted_offers:Dict(candidate_0:Discrete(2)), candidate_strengths:Dict(candidate_0:Discrete(101)), counter_offers:Dict(candidate_0:Tuple(Discrete(101), Discrete(11))), declined_offers:Dict(candidate_0:Tuple(Discrete(2), Discrete(101))), job_applicants:Dict(candidate_0:Discrete(2)), outstanding_offers:Dict(candidate_0:Tuple(Discrete(101), Discrete(11))), rejected_offers:Dict(candidate_0:Tuple(Discrete(2), Discrete(101))), remaining_budget:Discrete(101)))


In [17]:
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 5000000},
    checkpoint_freq=10,
    local_dir="~/ray_results/" + env_name,
    config=config,
)

(pid=1671497) 
(PPO pid=1671497) 2022-12-12 21:59:50,242	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=1671528) 
(pid=1671527) 
(RolloutWorker pid=1671528) E1212 21:59:59.126686409 1671528 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=1671528) /home/accts/ahc49/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
(RolloutWorker pid=1671528)   warnings.warn("Can't initialize NVML")
(RolloutWorker pid=1671528) E1212 21:59:59.151871834 1671528 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=1671528) E1212 21:59:59.169778629 1671528 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=1671528)

(PPO pid=1671497) E1212 21:59:59.390884965 1671497 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPO pid=1671497) /home/accts/ahc49/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
(PPO pid=1671497)   warnings.warn("Can't initialize NVML")
(PPO pid=1671497) E1212 21:59:59.412337936 1671497 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=1671528) 2022-12-12 21:59:59,350	INFO policy.py:1134 -- Policy (worker=2) running on CPU.
(RolloutWorker pid=1671528) 2022-12-12 21:59:59,350	INFO torch_policy_v2.py:101 -- Found 0 visible cuda devices.
(RolloutWorker pid=1671528) 2022-12-12 21:59:59,353	DEBUG preprocessors.py:270 -- Creating sub-preprocessor for Dict(accepted_offer:Dict(employer_0:Discrete(101)), counter_offers:Dict(employer_0:Tuple(Discrete(101), Discrete(11))), current_offers:Dict(employer_0:Tuple(Discr

(PPO pid=1671497) 2022-12-12 21:59:59,535	DEBUG rollout_worker.py:1864 -- Creating policy for default_policy
(PPO pid=1671497) 2022-12-12 21:59:59,537	DEBUG preprocessors.py:270 -- Creating sub-preprocessor for Dict(accepted_offer:Dict(employer_0:Discrete(101)), counter_offers:Dict(employer_0:Tuple(Discrete(101), Discrete(11))), current_offers:Dict(employer_0:Tuple(Discrete(101), Discrete(11))), job_openings:Dict(employer_0:Discrete(2)), rejected_offers:Dict(employer_0:Tuple(Discrete(2), Discrete(101))))
(PPO pid=1671497) 2022-12-12 21:59:59,538	DEBUG preprocessors.py:270 -- Creating sub-preprocessor for Dict(employer_0:Discrete(101))
(PPO pid=1671497) 2022-12-12 21:59:59,538	DEBUG preprocessors.py:270 -- Creating sub-preprocessor for Discrete(101)
(PPO pid=1671497) 2022-12-12 21:59:59,538	DEBUG preprocessors.py:270 -- Creating sub-preprocessor for Dict(employer_0:Tuple(Discrete(101), Discrete(11)))
(PPO pid=1671497) 2022-12-12 21:59:59,538	DEBUG preprocessors.py:270 -- Creating sub-pr

2022-12-12 21:59:59,682	ERROR trial_runner.py:993 -- Trial PPO_job_search_30f4a_00000: Error processing event.
ray.exceptions.RayTaskError(ValueError): ray::PPO.train() (pid=1671497, ip=128.36.232.24, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 355, in train
    raise skipped from exception_cause(skipped)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 352, in train
    result = self.step()
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 772, in step
    results, train_iter_ctx = self._run_one_training_iteration()
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 2953, in _run_one_training_iteration
    num_recreated += self.try_recover_from_step_attempt(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 2617,

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
PPO_job_search_30f4a_00000,2022-12-12_21-59-59,597ded83e9b54d668ceba359cf001b6d,swan.zoo.yale.edu,128.36.232.24,1671497,1670900399,30f4a_00000


(PPO pid=1671497) 2022-12-12 21:59:59,653	WARNING algorithm.py:2610 -- Worker crashed during training or evaluation! To try to continue without failed worker(s), set `ignore_worker_failures=True`. To try to recover the failed worker(s), set `recreate_failed_workers=True`.


TuneError: ('Trials did not complete', [PPO_job_search_30f4a_00000])

# Run 1
Got the error: 
```
AssertionError: Observation spaces for all agents must be identical. Perhaps SuperSuit's pad_observations wrapper can help (useage: `supersuit.aec_wrappers.pad_observations(env)`
```

# Run 2

```
AssertionError: homogenization only supports Discrete and Box spaces
```

# Run 3

Same error
```
Traceback (most recent call last):
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=1879526, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 139, in __init__
    self.add_workers(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 490, in add_workers
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 620, in foreach_worker
    remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=1879573, ip=128.36.108.57, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fa145f35d50>)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 492, in __init__
    self.env = env_creator(copy.deepcopy(self.env_context))
  File "/tmp/ipykernel_1861406/2067092441.py", line 2, in <lambda>
  File "/tmp/ipykernel_1861406/841127696.py", line 3, in env_creator
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/supersuit/multiagent_wrappers/padding_wrappers.py", line 33, in pad_observations_v0
    homogenize_ops.check_homogenize_spaces(spaces)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/supersuit/utils/action_transforms/homogenize_ops.py", line 30, in check_homogenize_spaces
    assert False, "homogenization only supports Discrete and Box spaces"
AssertionError: homogenization only supports Discrete and Box spaces

During handling of the above exception, another exception occurred:

ray::PPO.__init__() (pid=1879526, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 549, in setup
    raise e.args[0].args[2]
AssertionError: homogenization only supports Discrete and Box spaces
```

i.e. I cannot use the SuperSuit wrapper to fix the issue of observation spaces for all agents needing to be identical.

# Run 4

Fixed the issue by making all observation spaces and action spaces the same for all agents.

New issue:

```
Failure # 1 (occurred at 2022-12-12_17-37-17)
Traceback (most recent call last):
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=499392, ip=128.36.232.21, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 139, in __init__
    self.add_workers(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 490, in add_workers
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 620, in foreach_worker
    remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=499452, ip=128.36.232.21, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f8059b28df0>)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 567, in __init__
    self.policy_dict = _determine_spaces_for_multi_agent_dict(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 2121, in _determine_spaces_for_multi_agent_dict
    raise ValueError(
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!

During handling of the above exception, another exception occurred:

ray::PPO.__init__() (pid=499392, ip=128.36.232.21, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 549, in setup
    raise e.args[0].args[2]
ValueError: `observation_space` not provided in PolicySpec for default_policy and env does not have an observation space OR no spaces received from other workers' env(s) OR no `observation_space` specified in config!
```

# Run 5

```
Failure # 1 (occurred at 2022-12-12_20-29-44)
Traceback (most recent call last):
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/execution/ray_trial_executor.py", line 1050, in get_next_executor_event
    future_result = ray.get(ready_future)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/_private/worker.py", line 2291, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=2054378, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 139, in __init__
    self.add_workers(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 490, in add_workers
    self.foreach_worker(lambda w: w.assert_healthy())
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/worker_set.py", line 620, in foreach_worker
    remote_results = ray.get([w.apply.remote(func) for w in self.remote_workers()])
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=2054502, ip=128.36.108.57, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f0abd339d50>)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 625, in __init__
    self._build_policy_map(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1882, in _build_policy_map
    preprocessor = ModelCatalog.get_preprocessor_for_space(
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/models/catalog.py", line 815, in get_preprocessor_for_space
    prep = cls(observation_space, options)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/models/preprocessors.py", line 42, in __init__
    self._size = int(np.product(self.shape))
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

During handling of the above exception, another exception occurred:

ray::PPO.__init__() (pid=2054378, ip=128.36.108.57, repr=PPO)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 414, in __init__
    super().__init__(config=config, logger_creator=logger_creator, **kwargs)
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 161, in __init__
    self.setup(copy.deepcopy(self.config))
  File "/home/accts/ahc49/.local/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py", line 549, in setup
    raise e.args[0].args[2]
TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'
```

Note: the `Dict` space does not have a shape <https://github.com/openai/gym/blob/master/gym/spaces/dict.py#L118>

In [32]:
# HOLY SHIT, use gym.space instead of gymnasium.spaces to fix this fucking bullshit AHHHHHHHH

from gym.spaces import Discrete, Dict, Tuple

from gym.spaces.utils import flatten, flatdim

In [28]:
print(Tuple((Discrete(3), Discrete(2))).shape)

None


In [29]:
Discrete(3).shape

()

In [30]:
from ray.rllib.models.preprocessors import DictFlatteningPreprocessor, get_preprocessor

In [31]:
obs_space = Dict({"test": Discrete(3)})
dfpp = DictFlatteningPreprocessor(obs_space)

In [49]:
print(dfpp)

In [50]:
get_preprocessor(Dict())

ray.rllib.models.preprocessors.DictFlatteningPreprocessor